In [1]:
import pandas as pd

In [2]:
data = {
    'day': ['Monday']*6 + ['Tuesday']*6,
    'product': ['apple', 'banana', 'carrot', 'broccoli', 'pork', 'beef']*2,
    'food_type': ['fruit', 'fruit', 'vegetable', 'vegetable', 'meat', 'meat']*2,
    'amount_sold': [200, 50, 300, 75, 100, 50] + [400, 100, 600, 150, 200, 100],
    'sales': [100, 50, 60, 25, 200, 150] + [200, 100, 120, 50, 400, 300]
}
df = pd.DataFrame(data)
df

,day,product,food_type,amount_sold,sales
0,Monday,apple,fruit,200,100
1,Monday,banana,fruit,50,50
2,Monday,carrot,vegetable,300,60
3,Monday,broccoli,vegetable,75,25
4,Monday,pork,meat,100,200
5,Monday,beef,meat,50,150
6,Tuesday,apple,fruit,400,200
7,Tuesday,banana,fruit,100,100
8,Tuesday,carrot,vegetable,600,120
9,Tuesday,broccoli,vegetable,150,50


# pivot_table
- pivot_table uses an aggregate function, pivot does not

In [3]:
df.pivot_table(index='product', columns='day', values='sales', aggfunc='sum')

day,Monday,Tuesday
product,,
apple,100,200
banana,50,100
beef,150,300
broccoli,25,50
carrot,60,120
pork,200,400


In [4]:
df.pivot_table(index='day', columns='food_type', values='sales')

food_type,fruit,meat,vegetable
day,,,
Monday,75.0,175.0,42.5
Tuesday,150.0,350.0,85.0


In [5]:
df.pivot_table(index='day', columns='food_type', values='sales', aggfunc='sum') # aggfunc defaults to 'mean'

food_type,fruit,meat,vegetable
day,,,
Monday,150,350,85
Tuesday,300,700,170


In [6]:
df.pivot_table(columns='day', aggfunc='max', values=['amount_sold', 'sales'])

day,Monday,Tuesday
amount_sold,300,600
sales,200,400


# pivot

- pivot_table uses an aggregate function, pivot does not
- for pivot, column-index combos must be unique

In [7]:
df.pivot(index='product', columns='day', values=['sales'])

sales        
day      Monday Tuesday
product                
apple       100     200
banana       50     100
beef        150     300
broccoli     25      50
carrot       60     120
pork        200     400

# melt
- use melt to go from wide to skinny
- can also think of it as un-pivoting

In [8]:
wide_df = df.pivot(index='day', columns='product', values='sales').reset_index()
wide_df.columns.name  = None
wide_df

,day,apple,banana,beef,broccoli,carrot,pork
0,Monday,100,50,150,25,60,200
1,Tuesday,200,100,300,50,120,400


In [9]:
wide_df.melt(id_vars='day', var_name='product', value_name='sales')

,day,product,sales
0,Monday,apple,100
1,Tuesday,apple,200
2,Monday,banana,50
3,Tuesday,banana,100
4,Monday,beef,150
5,Tuesday,beef,300
6,Monday,broccoli,25
7,Tuesday,broccoli,50
8,Monday,carrot,60
9,Tuesday,carrot,120


# stack

In [10]:
pdf = df.pivot(index='product', columns='day')
pdf

food_type            amount_sold          sales        
day          Monday    Tuesday      Monday Tuesday Monday Tuesday
product                                                          
apple         fruit      fruit         200     400    100     200
banana        fruit      fruit          50     100     50     100
beef           meat       meat          50     100    150     300
broccoli  vegetable  vegetable          75     150     25      50
carrot    vegetable  vegetable         300     600     60     120
pork           meat       meat         100     200    200     400

In [11]:
# level kwarg in stack refers to levels of multi-index
s = f'''
Column Object Type: {type(pdf.columns)}
Levels: {pdf.columns.levels}
First Level: {pdf.columns.levels[0]}
Second Level: {pdf.columns.levels[1]}
'''
print(s)


Column Object Type: <class 'pandas.core.indexes.multi.MultiIndex'>
Levels: [['food_type', 'amount_sold', 'sales'], ['Monday', 'Tuesday']]
First Level: Index(['food_type', 'amount_sold', 'sales'], dtype='object')
Second Level: Index(['Monday', 'Tuesday'], dtype='object', name='day')



In [12]:
# stacking level 0 takes the first level from column MuliIndex object and pivots it to inner-most level of index
pdf.stack(level=0)

day                      Monday    Tuesday
product                                   
apple    amount_sold        200        400
         food_type        fruit      fruit
         sales              100        200
banana   amount_sold         50        100
         food_type        fruit      fruit
         sales               50        100
beef     amount_sold         50        100
         food_type         meat       meat
         sales              150        300
broccoli amount_sold         75        150
         food_type    vegetable  vegetable
         sales               25         50
carrot   amount_sold        300        600
         food_type    vegetable  vegetable
         sales               60        120
pork     amount_sold        100        200
         food_type         meat       meat
         sales              200        400

In [13]:
# stacking level 1 takes the second level from column MuliIndex object and pivots it to inner-most level of index
pdf.stack(level=1)

food_type  amount_sold  sales
product  day                                   
apple    Monday       fruit          200    100
         Tuesday      fruit          400    200
banana   Monday       fruit           50     50
         Tuesday      fruit          100    100
beef     Monday        meat           50    150
         Tuesday       meat          100    300
broccoli Monday   vegetable           75     25
         Tuesday  vegetable          150     50
carrot   Monday   vegetable          300     60
         Tuesday  vegetable          600    120
pork     Monday        meat          100    200
         Tuesday       meat          200    400

In [14]:
df.head(2)

,day,product,food_type,amount_sold,sales
0,Monday,apple,fruit,200,100
1,Monday,banana,fruit,50,50


In [15]:
# Stack level defaults to -1 (inner most column level) 
# calling stack on single level columns returns an unnamed MultiIndex series with columns pivoted to be inner-most level of index
df.stack().head(10)

0  day            Monday
   product         apple
   food_type       fruit
   amount_sold       200
   sales             100
1  day            Monday
   product        banana
   food_type       fruit
   amount_sold        50
   sales              50
dtype: object

# unstack

In [16]:
# multi-index dataframe
midf = pdf.stack(level=0)
midf.head(6)

day                 Monday Tuesday
product                           
apple   amount_sold    200     400
        food_type    fruit   fruit
        sales          100     200
banana  amount_sold     50     100
        food_type    fruit   fruit
        sales           50     100

In [17]:
# unstack pivots a level of the index to the inner-most level of the columns
midf.unstack(level=-1) # defaults to level=-1

day           Monday                      Tuesday                 
         amount_sold  food_type sales amount_sold  food_type sales
product                                                           
apple            200      fruit   100         400      fruit   200
banana            50      fruit    50         100      fruit   100
beef              50       meat   150         100       meat   300
broccoli          75  vegetable    25         150  vegetable    50
carrot           300  vegetable    60         600  vegetable   120
pork             100       meat   200         200       meat   400